#AL Week 3 exercise


---

*Aditi Desai*

*200968126*

The main function in this program first loads data into memory (the directory from which  the  data  is  loaded  can  be  specified  by  a  command-line  argument).  Then,  the function  prompts the user  to type  in  two  names.  The  person_id_for_name  function retrieves the id for any person (and handles prompting the user to clarify, in the event that multiple people have the same name). The function then calls the shortest_path function  to  compute  the  shortest  path  between  the  two  people,  and  prints  out  the path.

The shortest_path function has to be coded using 

a.Breadth First Search

b.Depth First Search

In [1]:
#Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys 
import csv
import time

%matplotlib inline

In [2]:
df_movies=pd.read_csv('movies.csv')
df_people=pd.read_csv('people.csv')
df_stars=pd.read_csv('stars.csv')

In [3]:
df_movies.head()

,id,title,year
0,112384,Apollo 13,1995
1,104257,A Few Good Men,1992
2,109830,Forrest Gump,1994
3,93779,The Princess Bride,1987
4,95953,Rain Man,1988


In [4]:
df_people.head

<bound method NDFrame.head of         id             name  birth
0      102      Kevin Bacon   1958
1      129       Tom Cruise   1962
2      144       Cary Elwes   1962
3      158        Tom Hanks   1956
4     1597   Mandy Patinkin   1952
5      163   Dustin Hoffman   1937
6     1697   Chris Sarandon   1942
7      193       Demi Moore   1962
8      197   Jack Nicholson   1937
9      200      Bill Paxton   1955
10     398      Sally Field   1946
11     420   Valeria Golino   1965
12  596520  Gerald R. Molen   1935
13     641      Gary Sinise   1955
14     705     Robin Wright   1966
15  914612      Emma Watson   1990>

In [5]:
df_stars.head()

,person_id,movie_id
0,102,104257
1,102,112384
2,129,104257
3,129,95953
4,144,93779


In [6]:
#Mapping
names = {}
people = {}
movies = {}

In [7]:
#Load data
def load_data():
    # Load people
    with open(f"/content/people.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            people[row["id"]] = {
                "name": row["name"],
                "birth": row["birth"],
                "movies": set()
            }
            if row["name"].lower() not in names:
                names[row["name"].lower()] = {row["id"]}
            else:
                names[row["name"].lower()].add(row["id"])

    #Load movies
    with open(f"/content/movies.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            movies[row["id"]] = {
                "title": row["title"],
                "year": row["year"],
                "stars": set()
            }

    #Load stars
    with open(f"/content/stars.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            try:
                people[row["person_id"]]["movies"].add(row["movie_id"])
                movies[row["movie_id"]]["stars"].add(row["person_id"])
            except KeyError:
                pass

In [8]:
#Map person id and name
def person_id_for_name(name):
    person_ids = list(names.get(name.lower(), set()))
    if len(person_ids) == 0:
        return None
    elif len(person_ids) > 1:
        print(f"Which '{name}'?")
        for person_id in person_ids:
            person = people[person_id]
            name = person["name"]
            birth = person["birth"]
            print(f"ID: {person_id}, Name: {name}, Birth: {birth}")
        try:
            person_id = input("Intended Person ID: ")
            if person_id in person_ids:
                return person_id
        except ValueError:
            pass
        return None
    else:
        return person_ids[0]

In [27]:
!pip install utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [29]:
from utils import Node, StackFrontier, QueueFrontier

ImportError: ignored

In [9]:
#Return neighbours
def neighbors_for_person(person_id):
    movie_ids = people[person_id]["movies"]
    neighbors = set()
    for movie_id in movie_ids:
        for person_id in movies[movie_id]["stars"]:
            neighbors.add((movie_id, person_id))
    return neighbors


In [10]:
#Shortest path DFS
def shortest_path_dfs(source, target):
  print("DFS")
  print(f"Finding shortest path between {people[source]['name']} ({source}) and {people[target]['name']} ({target})...")
  timer = time.time()

  #Start with stack and initial node
  stack = StackFrontier()
  initial_node = Node(state=source, parent=None, action=None)
  stack.add(initial_node)

  #Start with empty explored set
  explored = set()
  number_of_states_explored = 0

  while True:
   #If stack is empty no solution
   if stack.empty():
        return None
    #Remove a node from the stack
   node = stack.remove()
   number_of_states_explored += 1

   #Add the node to the explored set
   explored.add(node.state)

   #Expand node, add resulting nodes to the stack if they aren't already in the  or the explored set
   for movie_id, person_id in neighbors_for_person(node.state):
      if not stack.contains_state(person_id) and person_id not in explored:
          child = Node(state=person_id, parent=node, action=movie_id)

          #If child node (neighbor) contains goal state, no need to add it to the stack instead return the solution immediately.
          if child.state == target:
            path = []
            node = child

            while node.parent is not None:
              path.append((node.action, node.state))
              node = node.parent
            path.reverse()

            seconds_taken = time.time() - timer
            print(f"Explored { number_of_states_explored } states in { seconds_taken } seconds")
                    
            return path

          stack.add(child)
    
  return None

  raise NotImplementedError

In [11]:
#Shortest path BFS
def shortest_path_bfs(source, target):
    print("BFS")
    print(f"Finding shortest path between {people[source]['name']} ({source}) and {people[target]['name']} ({target})...")
    timer = time.time()

    #Start with queue and initial node
    queue = QueueFrontier()
    initial_node = Node(state=source, parent=None, action=None)
    queue.add(initial_node)

    #Start with empty explored set
    explored = set()
    number_of_states_explored = 0

    while True:

        #If queue is empty no solution
        if queue.empty():
            return None

        #Remove a node from the queue
        node = queue.remove()
        number_of_states_explored += 1

        #Add the node to the explored set
        explored.add(node.state)

        #Expand node, add resulting nodes to the queue if the aren't already in the queue or the explored set
        for movie_id, person_id in neighbors_for_person(node.state):
            if not queue.contains_state(person_id) and person_id not in explored:
                child = Node(state=person_id, parent=node, action=movie_id)

                #If child node (neighbor) contains goal state, no need to add it to the queue instead return the solution immediately.
                if child.state == target:
                    path = []
                    node = child

                    while node.parent is not None:
                        path.append((node.action, node.state))
                        node = node.parent
                    path.reverse()

                    seconds_taken = time.time() - timer
                    print(f"Explored { number_of_states_explored } states in { seconds_taken } seconds")
                    
                    return path

                queue.add(child)

In [12]:
#Main function
def main():
    #Load data
    load_data()
    print("Data loaded.")

    source = person_id_for_name(input("Name: "))
    if source is None:
        sys.exit("Person not found.")
    target = person_id_for_name(input("Name: "))
    if target is None:
        sys.exit("Person not found.")

    pathd = shortest_path_dfs(source, target)
    pathb = shortest_path_bfs(source, target)

    if pathd is None:
        print("Not connected.")
    else:
        degrees = len(pathd)
        print(f"{degrees} degrees of separation.")
        pathd = [(None, source)] + pathd
        for i in range(degrees):
            person1 = people[pathd[i][1]]["name"]
            person2 = people[pathd[i + 1][1]]["name"]
            movie = movies[pathd[i + 1][0]]["title"]
            print(f"{i + 1}: {person1} and {person2} starred in {movie}")

    if pathb is None:
        print("Not connected.")
    else:
        degrees = len(pathb)
        print(f"{degrees} degrees of separation.")
        pathb = [(None, source)] + pathb
        for i in range(degrees):
            person1 = people[pathb[i][1]]["name"]
            person2 = people[pathb[i + 1][1]]["name"]
            movie = movies[pathb[i + 1][0]]["title"]
            print(f"{i + 1}: {person1} and {person2} starred in {movie}")

In [17]:
#Run
if __name__ == "__main__":
    main()

Data loaded.
Name: Tom Hanks
Name: Bill Paxton
DFS
Finding shortest path between Tom Hanks (158) and Bill Paxton (200)...


NameError: ignored